# Revised case normalization for Hirslanden Salem 2017

This jupyter notebook is used to normalize the revised case from DtoD.

Before runing the notebook, raw_data folder need to added to the root directory

The raw data folder can be find here: https://aimedic.sharepoint.com/:f:/s/dev/Ejx_A1dg8gtPumFknOWOh0oBi6ofx9hctYiq3c-0gH9vYA?e=UmcgrS

Normalization:

-  Convert the column names to the name used in the Database
-  Delete cases which is empty in the follow columns VALIDATION_COLS: 'case_id', 'patient_id', 'gender', 'age_years', duration_of_stay',  'pccl',  'drg'
- choose neccessary columns COLS_TO_SELECT: case_id, patient_id, gender, age_years, duration_of_stay, pccl, drg, pd, bfs_code, added_icds, removed_icds, added_chops, removed_chops
- still need to do (TODO):    
    -  Check CHOP upper/lowercase
    -  Check whether the PD changed. If it did, new and old PD are stored together with added and removed ICDs, respectively
    -  Pad case IDs with 0s
    -  Write function to validate cases


In [1]:
import pandas as pd
import os
from dataclasses import dataclass, field
import sys
sys.path.insert(0, '/home/jovyan/work')
sys.path.insert(1, '/home/jovyan/work/src')
sys.path.insert(2, '/home/jovyan/work/src/service')

from service import bfs_cases_db_service as bfs_db

from py.global_configs import *
from py.normalize import normalize

/home/jovyan/work/src/models/BfsCase.py:5: SAWarning: On class 'BfsCase', Column object 'drg_cost_weight' named directly multiple times, only one will be used: drg_cost_weight, effective_cost_weight. Consider using orm.synonym instead
  class BfsCase(Base):


In [2]:
# check all the file name

FILES_TO_ANALYZE.keys()


dict_keys(['Hirslanden Salem 2017', 'Hirslanden Beau Site 2017', 'Hirslanden Linde 2017', 'Hirslanden Linde 2018', 'Hirslanden Salem 2018', 'Hirslanden Beau Site 2018'])

In [3]:
file = FILES_TO_ANALYZE['Hirslanden Salem 2017']
file

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx', hospital_name_db='Hirslanden Salem', year='2017', sheets=['Änderungen_SA_2017'])

In [4]:

df_revised_case_d2d = normalize(file, 0)

Read 57 cases for Hirslanden Salem 2017
TYPES:
case_id             string
patient_id          string
gender              string
age_years            int64
duration_of_stay     int64
pccl                 int64
drg                 string
pd                  string
bfs_code            string
added_icds          string
removed_icds        string
added_chops         string
removed_chops       string
dtype: object


In [5]:
df_revised_case_d2d.head()

,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops
0,41194601,8A6B7DE0296BEFB8,W,73,10,3,I09C,M4806,M200,N183,<NA>,948x40::20170128,948X40::20170128
1,41248206,8DAB6117ADBB2AFE,M,84,8,3,I43B,M179,M200,I420,<NA>,99b812::20170212,99B812::20170212
2,41302785,5865672002F07F3F,W,97,13,3,F62C,I5001,M100,"E876,A099",<NA>,<NA>,<NA>
3,41304308,0CD7621F494A9140,M,36,3,3,L63D,N390,M100,"G8202,Q051",<NA>,<NA>,399521::20170304
4,41237114,376800437C6F8D5B,M,78,7,3,I09D,M9963,M200,"T842,F101",<NA>,<NA>,<NA>


# match to the database


In [6]:
# get the case_id from revised_case

revised_case_id = df_revised_case_d2d['case_id'].values
revised_case_id

array(['41194601', '41248206', '41302785', '41304308', '41237114',
       '41253912', '41255088', '41292119', '41338818', '41349287',
       '41364918', '41389850', '41389983', '41375575', '41405774',
       '41407198', '41417724', '41422903', '41429357', '41438489',
       '41442434', '41475362', '41397953', '41433914', '41437552',
       '41447097', '41230075', '41506483', '41506527', '41506722',
       '41529453', '41456079', '41470794', '41487802', '41271168',
       '41300833', '41418204', '41420840', '41579881', '41581364',
       '41608546', '41367724', '41451537', '41464045', '41490828',
       '41597280', '41400942', '41409001', '41443673', '41452311',
       '41486224', '41577457', '41478672', '41502799', '41510657',
       '41495658', '41532546'], dtype=object)

In [7]:
# match to the database
revised_case_db = bfs_db.get_bfs_cases_by_ids(revised_case_id)
revised_case_db.head()

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,ventilation_hours,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl


In [8]:
# 
print('There are {} out of {} revised cases from DtoD that are matched with the database for {} {}'.format(len(revised_case_db), len(revised_case_id), file.hospital_name_db, file.year))

There are 0 out of 57 revised cases from DtoD that are matched with the database for Hirslanden Salem 2017


In [9]:
# if we find the match cases, then we need to check if the case_id, gender, year....are match